In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import pandas as pd

 # 计算中心发中心频次对应时效标准

In [ ]:
# 读取文件路径
file_path = r"D:\Timeliness\ExcelData\TaotianStandard\频次报表_20260117_202902.csv"

# 读取数据
frequency = pd.read_csv(file_path)

In [ ]:
# 需求的列
col_need = ['频次名称', "始发中心", "状态", "目的中心", "始发发车时间", "目的到车时间", "总运行时效"]
df_fq = frequency.copy().loc[:, col_need]

In [ ]:
df_fq.info()

In [ ]:
for t in ["始发发车时间", "目的到车时间"]:
    df_fq[t] = pd.to_datetime(df_fq[t], format="mixed")
display(df_fq[["始发发车时间", "目的到车时间"]].head(10))

In [ ]:
time_s = pd.to_datetime("2026-01-18 17:00:00", format="mixed")

display(df_fq['频次名称'].head(5))
df_fq['频次名称'] = df_fq['频次名称'].str.rsplit("-", n=1).str[0]
display(df_fq['频次名称'].unique())

In [ ]:
df_fq1 = df_fq.copy()
df_list: list[pd.DataFrame] = []
for name in df_fq['频次名称'].unique():
    df = df_fq1.loc[df_fq1['频次名称'] == name, :]
    
    if df['始发发车时间'].max() >= time_s:
        mask0 = df['始发发车时间'] < time_s
        df1 = df.loc[mask0, :]
        
        if not df1.empty: 
            mask1 = df1['始发发车时间'] == df1['始发发车时间'].max()
            df_list.append(df1.loc[mask1, :])
        else:
            mask1 = df['始发发车时间'] == df['始发发车时间'].max()
            df_list.append(df.loc[mask1, :])
    
    else:
        mask0 = df['始发发车时间'] == df['始发发车时间'].max()
        df_list.append(df.loc[mask0, :])

df_fq2 = pd.concat(df_list, axis=0)
print(df_fq2.shape[0])
display(df_fq2['频次名称'].unique())

In [ ]:
mask = df_fq2['频次名称'].duplicated(keep=False)
display(df_fq2.loc[mask, :].sort_values(by='频次名称'))


In [ ]:
df_fq2 = df_fq2.sort_values(by=["频次名称", "始发发车时间", "总运行时效"], ascending=False)
df_fq2 = df_fq2.drop_duplicates(subset="频次名称", keep="first")
print(df_fq2.shape[0])
display(df_fq2['频次名称'].unique())

In [ ]:
df_fq2['当天分钟数'] = (24 - df_fq2["始发发车时间"].dt.hour) * 60 - df_fq2["始发发车时间"].dt.minute

In [ ]:
# 区分早上发车和晚上发车
mask_mor = df_fq2["始发发车时间"] < time_s
df_morning = df_fq2.loc[mask_mor, :].sort_values(by="始发发车时间", ascending=False)
mask_night = df_fq2['始发发车时间'] >= time_s
df_night = df_fq2.loc[mask_night, :].sort_values(by="始发发车时间", ascending=True)
display(df_morning.head(10))
display(df_night.head(10))

In [ ]:
# 计算早上发车的时效标准
mor = df_morning.copy()
mor['分钟差值'] = mor['总运行时效'] - mor["当天分钟数"]

def mor_judge(x):
    if x < 0:
        return 1
    else:
        return 2 + x // (24*60)
mor['时效标准'] = mor['分钟差值'].apply(mor_judge)
display(mor.sort_values(by="分钟差值", ascending=False).head(10))

In [ ]:
night = df_night.copy()
night['分钟差值'] = night['总运行时效'] - night["当天分钟数"]

def night_judge(x):
    if x < 0:
        return 1
    else:
        return 1 + x // (24*60)
night['时效标准'] = night['分钟差值'].apply(night_judge)
display(night.sort_values(by="分钟差值", ascending=False).head(10))

In [ ]:
df_result = pd.concat([mor, night], axis=0)
for t in ["始发发车时间", "目的到车时间"]:
    df_result[t] = df_result[t].dt.strftime("%H: %M")
df_result.to_excel(r"D:\Timeliness\ExcelData\TaotianStandard\中心发中心-频次对应时效标准.xlsx", index=False)

# 关联平台标准的城市与中心

In [ ]:
path1 = r"D:\Timeliness\CsvData\TaotianStandard\城市对应中心基础表1119.csv"
path2 = r"D:\Timeliness\CsvData\TaotianStandard\平台时效标准.csv"

tt_city = pd.read_csv(path2)
center = pd.read_csv(path1)

display(tt_city.info())
display(center.info())

In [ ]:
city_to_center = center.set_index('城市')['发货中心'].to_dict()

# 使用 map 向量化映射（无循环）
tt_city['发货中心'] = tt_city['发货城市'].map(city_to_center)
tt_city['签收中心'] = tt_city['签收城市'].map(city_to_center)

tt_city['中心线路'] = tt_city['发货中心'] + "-" + tt_city['签收中心']

display(tt_city.head(10))

In [ ]:
path = r"D:\Timeliness\ExcelData\TaotianStandard\平台时效标准-中心线路.xlsx"
tt_city.to_excel(path, index=False)

# 匹配中心当前频次标准和平台标准

In [ ]:
path1 = r"D:\Timeliness\CsvData\TaotianStandard\平台时效标准-中心线路.csv"
path2 = r"D:\Timeliness\CsvData\TaotianStandard\中心发中心-频次对应时效标准.csv"

platform = pd.read_csv(path1)
center = pd.read_csv(path2)

display(platform.info())
display(center.info())

In [ ]:
pf_need = ['发货省份', "发货城市", "发货中心", "收货省份", "签收城市", "签收中心", "中心线路", "淘天标准"]
center_need = ['频次名称', '时效标准', '始发发车时间', '目的到车时间', '总运行时效']

platform = platform.loc[:, pf_need]
center = center.loc[:, center_need]
df = pd.merge(platform, center, how='left', left_on = "中心线路", right_on="频次名称")
display(df.head(10))

In [67]:
mask = (df['淘天标准'] != "-") & (df['淘天标准'].notna())
df1 = df.copy().loc[mask, :]
df1['淘天标准'] = df1['淘天标准'].astype(float)
print(df1['时效标准'].dtype)
print(df1['淘天标准'].dtype)

float64
float64


In [70]:
df1["标准差值"] = df1['时效标准'] - df1['淘天标准']

df2 = df.copy().loc[~mask, :]

df_result = pd.concat([df1, df2], axis=0)
df_result = df_result.sort_values(by="标准差值", ascending=False)
display(df_result.head(10))

,发货省份,发货城市,发货中心,收货省份,签收城市,签收中心,中心线路,淘天标准,频次名称,时效标准,始发发车时间,目的到车时间,总运行时效,标准差值
50360,浙江省,金华市,义乌,广东省,清远市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
81958,浙江省,衢州市,义乌,广东省,云浮市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
81970,浙江省,衢州市,义乌,广东省,肇庆市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
58900,浙江省,丽水市,义乌,广东省,云浮市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
58778,浙江省,丽水市,义乌,广东省,清远市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
81702,浙江省,衢州市,义乌,广东省,广州市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
58799,浙江省,丽水市,义乌,广东省,韶关市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
50226,浙江省,金华市,义乌,广东省,广州市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
81857,浙江省,衢州市,义乌,广东省,韶关市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0
81836,浙江省,衢州市,义乌,广东省,清远市,广州,义乌-广州,2.0,义乌-广州,7.0,06: 00,12: 51,9045.0,5.0


In [71]:
path3 = r"D:\Timeliness\ExcelData\TaotianStandard\淘天平台标准.xlsx"
df_result.to_excel(path3, index=False)

# 计算网点交件班次差距

In [ ]:
path1 = r"D:\Timeliness\CsvData\TaotianStandard\网点交件班次信息 (0116).csv"
